In [ ]:
# --- 1. Importar Librerías Esenciales ---

# Librerías para manipulación de datos
import pandas as pd
# Uso: Fundamental para manejar DataFrames (tus tablas de datos).
#      Se usará para cargar los CSV, manipular columnas, etc.

import numpy as np
# Uso: Para operaciones numéricas eficientes, especialmente con arrays de números.
#      Scikit-learn trabaja mucho con arrays de NumPy internamente.

import re
# Uso: Para expresiones regulares. Necesario para la función de extracción de 'Title' del nombre
#      y también para extraer el 'Ticket_Prefix' del número de billete.

# Módulos de Scikit-learn para la construcción del Pipeline y Modelado
from sklearn.model_selection import train_test_split, GridSearchCV
# train_test_split: Para dividir el dataset en conjuntos de entrenamiento y validación.
# Uso: Asegura que el modelo se evalúe en datos no vistos durante el entrenamiento.
# GridSearchCV: Para la optimización de hiperparámetros de nuestro modelo y/o pipeline.
# Uso: Busca sistemáticamente la mejor combinación de parámetros.

from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
# StandardScaler: Para estandarizar (escalar) las características numéricas.
# Uso: Transforma los datos para que tengan media 0 y desviación estándar 1, útil para muchos algoritmos.
# OneHotEncoder: Para convertir características categóricas (como 'Sex', 'Embarked') a formato numérico binario.
# Uso: Permite que los modelos de ML trabajen con datos categóricos.
# FunctionTransformer: Para integrar funciones personalizadas (como tus funciones de ingeniería de características)
# Uso: Te permite usar tus funciones Python dentro del pipeline de Scikit-learn.

from sklearn.impute import SimpleImputer
# Uso: Para manejar los valores faltantes. Por ejemplo, rellenar 'Age' o 'Fare' con la media.

from sklearn.compose import ColumnTransformer
# Uso: Permite aplicar diferentes transformaciones a diferentes columnas del DataFrame.
#      Es clave para nuestro pipeline, ya que tenemos columnas numéricas y categóricas.

from sklearn.pipeline import Pipeline
# Uso: La clase principal para construir tu pipeline. Encadena todos los pasos de preprocesamiento y el modelo.
#      Esto garantiza que todas las transformaciones se apliquen consistentemente.

from sklearn.ensemble import RandomForestClassifier
# Uso: El algoritmo de Machine Learning que usaremos para nuestro modelo de clasificación binaria.
#      Es un clasificador de conjunto robusto y de alto rendimiento.

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Uso: Para evaluar el rendimiento de nuestro modelo.
# accuracy_score: La métrica principal para el concurso de Kaggle.
# confusion_matrix: Muestra la cantidad de verdaderos positivos, negativos, falsos positivos y negativos.
# classification_report: Proporciona precisión (precision), recall (sensibilidad) y F1-score por clase.

# Librerías para visualización de datos (aunque en este notebook será más para mostrar resultados)
import matplotlib.pyplot as plt
import seaborn as sns
# Uso: Para crear gráficos, como la matriz de confusión, para visualizar y entender los resultados del modelo.

# Configuración para gráficos
sns.set_style("whitegrid")
plt.style.use("seaborn-v0_8-darkgrid")
# Uso: Ajustes estéticos para que los gráficos se vean bien.

In [ ]:
sns.set_style('whitegrid')
plt.style.use('seaborn-v0_8-darkgrid')

In [ ]:
df_train_raw = pd.read_csv('../data/raw/train.csv')
df_test_raw = pd.read_csv('../data/raw/test.csv')
df_join_raw = pd.concat([df_train_raw.drop('Survived', axis=1), df_test_raw], ignore_index=True)
temp_df = df_join_raw.copy()
print(f'Dimensiones iniciales: {df_join_raw.shape}')
print(f'Columnas iniciales: {df_join_raw.columns.to_list}')

In [ ]:
df_join_raw

In [ ]:

def get_title_from_name(df):
    df = df.copy()
    def extract_title(name):
        title_extracted = re.search(' ([A-Za-z]+)\.', name)
        if title_extracted:
            return title_extracted.group(1)
        return 'rare'
    df['Title'] = df['Name'].apply(extract_title)
    df = df.drop('Name', axis=1)
    return df


def get_deck_from_cabin(df):
    df = df.copy()
    df['Cabin'] = df['Cabin'].fillna('Unknown')
    df['Deck'] = df['Cabin'].transform(lambda x: x[0])
    df = df.drop('Cabin', axis=1)
    return df

def get_agency_ticket_numbers_from_ticket(df):
    df = df.copy()
    def extract_prefix(ticket):
        prefix_extracted = re.match(r'([A-Za-z\./]+)', ticket)
        if prefix_extracted:
            return prefix_extracted.group(1).replace('.', '').replace('/', '').upper()
        return 'NO_AGENCY'
    df['Agency'] = df['Ticket'].apply(extract_prefix)
    ticket_counts = df['Ticket'].value_counts()
    df['TicketNumber'] = df['Ticket'].map(ticket_counts)
    df = df.drop('Ticket', axis=1)
    return df

def familysize_isAlone_from_sibsp_parch(df):
    df = df.copy()
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    df = df.drop(['Parch', 'SibSp'], axis=1)
    return df


feature_engineering_pipeline = Pipeline([
    ('get_title', FunctionTransformer(get_title_from_name, validate=False)),
    ('get_deck', FunctionTransformer(get_deck_from_cabin, validate=False)),
    ('get_agency', FunctionTransformer(get_agency_ticket_numbers_from_ticket, validate=False)),
    ('FamilySize', FunctionTransformer(familysize_isAlone_from_sibsp_parch, validate=False))
])
print('Cargado transformadores de ingenieria de features')

In [ ]:
def reorgize_ticket_class(matrix_feature):
    temp_df = pd.DataFrame(matrix_feature, columns=['Pclass'])
    def condition_class(ticket_class):
        class_mapping = { 1: 2,  2: 1, 3: 0}
        return class_mapping.get(ticket_class, 0)
    temp_df['Pclass'] = temp_df['Pclass'].apply(condition_class)
    return temp_df['Pclass'].values.reshape(-1,1)
    
    

def age_imputed_from_title_pclass(matrix_features):
    temp_df = pd.DataFrame(matrix_features, columns=['Age', 'Pclass', 'Title'])
    temp_df['Age'] = temp_df.groupby(['Pclass', 'Title'])['Age'].transform(lambda x: x.fillna(x.median()))
    temp_df['Age'] = temp_df['Age'].fillna(temp_df['Age'].median())
    return temp_df['Age'].values.reshape(-1,1)

    
print('Cargado imputadores numericos y categoricos')

In [ ]:
numeric_features = ['Pclass', 'Age', 'Fare', 'TicketNumber', 'FamilySize']

numeric_transformer = Pipeline(steps=[
    ('inputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])
numeric_transformer.set_output(transform="pandas")

categorical_features = ['Embarked', 'Title', 'Deck', 'Agency', 'Sex']

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False))
])

categorical_transformer.set_output(transform="pandas")

preprocessor = ColumnTransformer(
    transformers=[
        ('custom_imputer_age', FunctionTransformer(age_imputed_from_title_pclass), ['Age', 'Pclass', 'Title']),
        ('custom_reorganize_pclass', FunctionTransformer(reorgize_ticket_class), ['Pclass']),
        ('numeric', numeric_transformer, numeric_features),
        ('categorical', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)
print('preprocessor created')

In [ ]:
full_pipeline = Pipeline(steps=[
    ('features_engineering', feature_engineering_pipeline),
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])
full_pipeline.set_output(transform="pandas")
print('defined full pipelines')

In [ ]:
X = df_train_raw.drop(['Survived'], axis=1)
y = df_train_raw['Survived']
X_test_final = df_test_raw.copy()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

In [ ]:
x_demos = full_pipeline.named_steps['features_engineering'].fit_transform(X_train)
x_demos = full_pipeline.named_steps['preprocessor'].fit_transform(x_demos)
print(x_demos.columns.tolist())

In [ ]:
full_pipeline.fit(X_train, y_train)
y_pred_val_pipeline = full_pipeline.predict(X_val)
acurrancy_pipeline = accuracy_score(y_val, y_pred_val_pipeline)
print(f'Accuracy en el conjunto de validacion {acurrancy_pipeline}')
print(confusion_matrix(y_val, y_pred_val_pipeline))


In [ ]:
final_test_predictions = full_pipeline.predict(X_test_final)
submission_df = pd.DataFrame({
    'PassengerId': df_test_raw['PassengerId'],
    'Survived': final_test_predictions
})
submission_df.to_csv('../data/result/submission.csv', index=False)